In [0]:
%restart_python



In [0]:
%pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
download_path ="/Volumes/workspace/default/netflix/kaggle"
dbutils.fs.mkdirs(download_path)

True

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "ankitahub"
os.environ['KAGGLE_KEY'] ="10ab0b2b8f256946825ceb12f5c30199"

In [0]:
!kaggle datasets download -d shivamb/netflix-shows -p {download_path} --unzip


Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0
  0%|                                               | 0.00/1.34M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.34M/1.34M [00:00<00:00, 6.86MB/s]


In [0]:
import pandas as pd
df = pd.read_csv(download_path+"/netflix_titles.csv")


In [0]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [0]:
df = df.drop(columns=["title", "director", "cast" , "show_id", "description"])

In [0]:
df

,type,country,date_added,release_year,rating,duration,listed_in
0,Movie,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries
1,TV Show,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,TV Show,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,TV Show,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV"
4,TV Show,India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."
...,...,...,...,...,...,...,...
8802,Movie,United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers"
8803,TV Show,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies"
8804,Movie,United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies"
8805,Movie,United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies"


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   country       7976 non-null   object
 2   date_added    8797 non-null   object
 3   release_year  8807 non-null   int64 
 4   rating        8803 non-null   object
 5   duration      8804 non-null   object
 6   listed_in     8807 non-null   object
dtypes: int64(1), object(6)
memory usage: 481.8+ KB


In [0]:
df = df.drop_duplicates()

In [0]:
df = df.rename(columns={
    "type": "content_type",
    "listed_in": "genres"
})

In [0]:
print("Missing values before cleaning:\n", df.isnull().sum())

Missing values before cleaning:
 content_type      0
country           0
date_added       10
release_year      0
rating            0
duration          0
genres            0
duration_int      3
duration_type     0
dtype: int64


In [0]:
df = df.fillna({
    
    "country": "Unknown",
    "rating": "Not Rated",
    "duration": "Unknown"
})


In [0]:
df['duration'] = df['duration'].str.replace("Seasons", "Season")  # unify plural
df['duration'] = df['duration'].str.strip()

In [0]:
df['duration_int'] = df['duration'].str.extract(r'(\d+)').astype(float)
df['duration_type'] = df['duration'].str.extract(r'([a-zA-Z]+)')

In [0]:
df['duration_int'] = df['duration_int'].astype('Int64')

In [0]:
df.drop_duplicates(inplace=True)

In [0]:
for col in ['content_type', 'country', 'rating', 'genres']:
    df[col] = df[col].astype(str).str.strip()

In [0]:
df.reset_index(drop=True, inplace=True)

In [0]:
print(df.head(10))
print(df[['duration', 'duration_int', 'duration_type']].sample(10))

  content_type  ... duration_type
0        Movie  ...           min
1      TV Show  ...        Season
2      TV Show  ...        Season
3      TV Show  ...        Season
4      TV Show  ...        Season
5      TV Show  ...        Season
6        Movie  ...           min
7        Movie  ...           min
8      TV Show  ...        Season
9        Movie  ...           min

[10 rows x 9 columns]
      duration  duration_int duration_type
3428   100 min           100           min
2584  7 Season             7        Season
6175    78 min            78           min
6567    90 min            90           min
6626  1 Season             1        Season
4482   150 min           150           min
8380    93 min            93           min
3805    94 min            94           min
3888    95 min            95           min
6753    72 min            72           min


In [0]:
df

,content_type,country,date_added,release_year,rating,duration,genres,duration_int,duration_type
0,Movie,United States,25-09-21,2020,PG-13,90 min,Documentaries,90,min
1,TV Show,South Africa,24-09-21,2021,TV-MA,2 Season,"International TV Shows, TV Dramas, TV Mysteries",2,Season
2,TV Show,Unknown,24-09-21,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",1,Season
3,TV Show,Unknown,24-09-21,2021,TV-MA,1 Season,"Docuseries, Reality TV",1,Season
4,TV Show,India,24-09-21,2021,TV-MA,2 Season,"International TV Shows, Romantic TV Shows, TV ...",2,Season
...,...,...,...,...,...,...,...,...,...
8730,Movie,United States,20-11-19,2007,R,158 min,"Cult Movies, Dramas, Thrillers",158,min
8731,TV Show,Unknown,01-07-19,2018,TV-Y7,2 Season,"Kids' TV, Korean TV Shows, TV Comedies",2,Season
8732,Movie,United States,01-11-19,2009,R,88 min,"Comedies, Horror Movies",88,min
8733,Movie,United States,11-01-20,2006,PG,88 min,"Children & Family Movies, Comedies",88,min


In [0]:
# download_path = "/Volumes/workspace/default/netflix/cleaned_data"

df.to_csv(f"{download_path}/netflix_newcleaned.csv", index=False)



In [0]:
# NORMALISATION

In [0]:

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xffd459944c20>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.12/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


In [0]:
df = pd.read_csv(f"{download_path}/netflix_newcleaned.csv")


In [0]:
# Categorical Normalization
# content_type → Label Encoded (Movie=0, TV Show=1)
# country → Frequency Encoded (# of shows/movies from that country)
# rating → Ordinal Encoded (G=0 … TV-MA=10, unknown = -1)
# genres → Frequency Encoded (based on main/first genre)
# duration_type → Label Encoded (min=0, Season=1, etc.)

# 🔹 Numeric Normalization
# release_year → Frequency Encoded (# of titles released that year)
# duration_int → Min-Max Normalized (scaled between 0–1)
# date_added → Converted to numeric (# of days since earliest date, normalized)

In [0]:
df_norm = df.copy()

In [0]:
le_content = LabelEncoder()
df_norm["content_type"] = le_content.fit_transform(df_norm["content_type"])

In [0]:
country_freq = df_norm["country"].value_counts().to_dict()
df_norm["country"] = df_norm["country"].map(country_freq)

In [0]:
rating_order = [
    "G", "PG", "PG-13", "R", "NC-17",   # Movies
    "TV-Y", "TV-Y7", "TV-G", "TV-PG", "TV-14", "TV-MA" # TV Shows
]
rating_mapping = {rating: idx for idx, rating in enumerate(rating_order)}
df_norm["rating"] = df_norm["rating"].map(rating_mapping).fillna(-1).astype(int)

In [0]:
df_norm["main_genre"] = df_norm["genres"].apply(lambda x: x.split(",")[0].strip())
genre_freq = df_norm["main_genre"].value_counts().to_dict()
df_norm["genres"] = df_norm["main_genre"].map(genre_freq)
df_norm.drop(columns=["main_genre"], inplace=True)

In [0]:
le_duration = LabelEncoder()
df_norm["duration_type"] = le_duration.fit_transform(df_norm["duration_type"])

In [0]:
year_freq = df_norm["release_year"].value_counts().to_dict()
df_norm["release_year"] = df_norm["release_year"].map(year_freq)

In [0]:
scaler = MinMaxScaler()
df_norm["duration_int"] = scaler.fit_transform(df_norm[["duration_int"]])

In [0]:
df_norm["date_added"] = pd.to_datetime(df_norm["date_added"], errors="coerce")
min_date = df_norm["date_added"].min()
df_norm["date_added"] = (df_norm["date_added"] - min_date).dt.days
df_norm["date_added"] = df_norm["date_added"].fillna(df_norm["date_added"].median())

In [0]:
df_norm

,content_type,country,date_added,release_year,rating,duration,genres,duration_int,duration_type
0,0,2805,5016.0,950,2,90 min,818,0.286174,2
1,1,30,5015.0,586,10,2 Season,749,0.003215,0
2,1,807,5015.0,586,10,1 Season,399,0.000000,0
3,1,807,5015.0,586,10,1 Season,220,0.000000,0
4,1,968,5015.0,586,10,2 Season,749,0.003215,0
...,...,...,...,...,...,...,...,...,...
8730,0,2805,4341.0,88,3,158 min,12,0.504823,2
8731,1,807,4024.0,1141,6,2 Season,386,0.003215,0
8732,0,2805,4028.0,152,3,88 min,1206,0.279743,2
8733,0,2805,4688.0,96,1,88 min,595,0.279743,2


In [0]:
df_norm.to_csv(f"{download_path}/netflix_normalized_full.csv", index=False)